# MLB Roster Optimization for 2025 Season

This notebook uses Mixed Integer Programming (MIP) to build an optimal MLB roster that:
- **Maximizes total projected WAR (xWAR)** for 2025
- **Stays within a salary budget**
- **Satisfies positional requirements**

## Prerequisites

Before running this notebook, you must:

1. **Train the models** by running these notebooks (in order):
   - `hitter_war_t_to_tp1_multiyear_backfill.ipynb` → saves `models/hitter_war_model.joblib`
   - `pitcher_war_t_to_tp1_multiyear_optimized.ipynb` → saves `models/pitcher_war_model.joblib`
   - `aav_regression.ipynb` → saves `models/hitter_salary_model.joblib` & `models/pitcher_salary_model.joblib`

2. **Prepare the player data** by running:
   ```bash
   python prepare_roster_data.py
   ```
   This generates `players.csv` with predicted 2025 xWAR and expected costs.

3. **Julia Requirements**: Julia with packages `CSV`, `DataFrames`, `JuMP`, and `Gurobi`


## 1) Load Julia Packages


In [91]:
using CSV
using DataFrames
using JuMP
using Gurobi


## 2) Load Player Data

Load the `players.csv` file generated by `prepare_roster_data.py`. This file contains:
- `player_id`: Unique MLB player ID
- `name`: Player name
- `position`: Player's primary position (C, 1B, 2B, 3B, SS, LF, CF, RF, DH, SP, RP)
- `xwar`: Predicted WAR/162 for 2025 season
- `cost`: Expected salary in millions of dollars


Additional optional columns supported:
- `market_cost`: **Actual market AAV** (in millions of $). Used to enforce an acceptance rule that we only sign players where our **predicted cost (`cost`) is greater than their actual market value** (`market_cost`).
- `locked`: Boolean flag (true/false) indicating players already on the **existing roster** who must be included. Locked players are always kept, even if they do not pass the value-gap filter, and are forced onto the roster in the optimization model.


In [92]:
# Load player data generated by prepare_roster_data.py
players_df = CSV.read("players.csv", DataFrame)

# Team selection: specify which team to optimize for
# Team selection: specify which team to optimize for
# Use full canonical team name (e.g., "New York Yankees", "New York Mets", "Los Angeles Dodgers")
target_team = "Boston Red Sox"  # Change this to your team


println("Loaded $(nrow(players_df)) players from players.csv")

# Expect columns: player_id, name, position, xwar, cost
required_cols = ["player_id", "name", "position", "xwar", "cost"]
@assert all(c -> c ∈ names(players_df), required_cols) "Missing required columns in players.csv"

# Filter out any players with negative xWAR (not valuable)
players_df = filter(row -> row.xwar > 0, players_df)
println("After filtering to positive xWAR: $(nrow(players_df)) players")

# Filter to target team players + all free agents
if "team_or_fa" ∈ names(players_df)
    initial_count = nrow(players_df)
    # Keep players from target team and all free agents
    players_df = filter(row -> row.team_or_fa == target_team || row.team_or_fa == "FA", players_df)
    filtered_count = nrow(players_df)
    println("After filtering to $target_team players + free agents: $filtered_count players (removed $(initial_count - filtered_count))")
    
    # Mark target team players as locked (incumbents)
    players_df.locked = [row.team_or_fa == target_team for row in eachrow(players_df)]
    incumbent_count = sum(players_df.locked)
    fa_count = sum(.!players_df.locked)
    println("  Incumbent players (locked): $incumbent_count")
    println("  Free agents available: $fa_count")
else
    println("Warning: No team_or_fa column found. Optimizing for all players.")
end


# Optional acceptance filter based on value gap between predicted cost and actual market value.
# Controlled by `apply_value_gap_filter` flag below.
# Decision rule: We cannot pay a free agent BELOW what they actually signed for.
# If `market_cost` is present and the filter is enabled, we keep:
#   - all locked players (incumbents), and
#   - any free agent where predicted cost (`cost`) >= actual market value (`market_cost`).
# This ensures we only consider free agents we can afford at their actual market rate.
# Controlled by `apply_value_gap_filter` flag below.
# If `market_cost` is present and the filter is enabled, we keep:
#   - all locked players (locked == true), and
#   - any free agent where predicted cost (`cost`) >= actual market value (`market_cost`).
apply_value_gap_filter = false  # set to true to require predicted cost >= market value

if apply_value_gap_filter && ("market_cost" ∈ names(players_df))
    has_locked = "locked" ∈ names(players_df)
    println("Detected column `market_cost` – applying value-gap filter (predicted cost >= market value).")
    players_df.value_gap = players_df.cost .- players_df.market_cost
    if has_locked
        println("Using `locked` column: locked players are always retained, even if value_gap < 0.")
        # Handle Missing values: locked players always kept, free agents need value_gap >= 0
        players_df = filter(row -> coalesce(row.locked, false) || coalesce(row.value_gap, -Inf) >= 0.0, players_df)
    else
        # Handle Missing values: only keep players where value_gap >= 0
        players_df = filter(row -> coalesce(row.value_gap, -Inf) >= 0.0, players_df)
    end
    println("After value-gap filter: $(nrow(players_df)) players")
elseif "market_cost" ∈ names(players_df)
    println("Detected `market_cost` column but value-gap filter disabled; keeping all players regardless of value_gap.")
else
    println("No `market_cost` column found – skipping value-gap acceptance filter.")
end

# Filter out players with missing market_cost if market_cost column exists
# (needed because JuMP cannot handle Missing values in constraints)
if "market_cost" ∈ names(players_df)
    initial_count = nrow(players_df)
    players_df = filter(row -> !ismissing(row.market_cost), players_df)
    dropped = initial_count - nrow(players_df)
    if dropped > 0
        println("Dropped $dropped players with missing market_cost (no contracts)")
    end
end


player_ids = unique(players_df.player_id)
positions  = unique(players_df.position)

# Optional: identify locked-in players (existing roster) if `locked` column is present.
if "locked" ∈ names(players_df)
    locked_players = [row.player_id for row in eachrow(players_df) if row.locked == true]
    println("Locked-in players (existing roster): $(length(locked_players))")
else
    locked_players = Any[]
    println("No `locked` column found – treating all players as free agents for the optimizer.")
end

# Maps for convenience
player_name = Dict(p => players_df.name[findfirst(==(p), players_df.player_id)]
                   for p in player_ids)
player_pos  = Dict(p => players_df.position[findfirst(==(p), players_df.player_id)]
                   for p in player_ids)
xwar        = Dict(p => players_df.xwar[findfirst(==(p), players_df.player_id)]
                   for p in player_ids)
cost        = Dict(p => players_df.cost[findfirst(==(p), players_df.player_id)]
                   for p in player_ids)


# If actual_war_2025 is available, build a dict for comparison
if "actual_war_2025" ∈ names(players_df)
    actual_war_2025 = Dict(p => players_df.actual_war_2025[findfirst(==(p), players_df.player_id)]
                       for p in player_ids)
else
    actual_war_2025 = Dict{Int, Union{Float64, Missing}}()
end

# If market_cost is available, build a parallel dict; otherwise fall back to `cost`.
if "market_cost" ∈ names(players_df)
    # Build market_cost dict, using cost as fallback for any missing values
    market_cost = Dict{Int, Float64}()
    for p in player_ids
        idx = findfirst(==(p), players_df.player_id)
        if idx !== nothing
            market_cost[p] = coalesce(players_df.market_cost[idx], cost[p])
        else
            market_cost[p] = cost[p]  # Fallback if player not found
        end
    end
else
    market_cost = cost

# IMPORTANT: For incumbent players (locked), market_cost MUST be their actual contract value.
# The budget constraint uses market_cost for all players, ensuring incumbents use real contracts.
end

# Players by position
players_by_pos = Dict{String, Vector{Int}}()
for r in positions
    players_by_pos[r] = [p for p in player_ids if player_pos[p] == r]
end

# Print position summary
println("\nPlayers by position:")
for pos in sort(collect(keys(players_by_pos)))
    println("  $pos: $(length(players_by_pos[pos])) players")
end

# Show top 5 players by xWAR
println("\nTop 10 players by xWAR:")
top_players = sort(player_ids, by=p -> -xwar[p])[1:min(10, length(player_ids))]
for p in top_players
    println("  $(rpad(player_name[p], 25)) $(rpad(player_pos[p], 4)) xWAR=$(round(xwar[p], digits=2))  \$$(round(cost[p], digits=2))M")
end


Loaded 1462 players from players.csv
After filtering to positive xWAR: 992 players
After filtering to Boston Red Sox players + free agents: 421 players (removed 571)
  Incumbent players (locked): 20
  Free agents available: 401
Detected `market_cost` column but value-gap filter disabled; keeping all players regardless of value_gap.
Dropped 148 players with missing market_cost (no contracts)
Locked-in players (existing roster): 20

Players by position:
  1B: 15 players
  2B: 19 players
  3B: 16 players
  C: 17 players
  CF: 12 players
  DH: 2 players
  LF: 16 players
  RF: 9 players
  RP: 100 players
  SP: 48 players
  SS: 14 players

Top 10 players by xWAR:
  Brayan Bello              SP   xWAR=10.0  $0.8M
  Taj Bradley               SP   xWAR=10.0  $0.64M
  Corbin Burnes             SP   xWAR=10.0  $5.9M
  Aaron Civale              SP   xWAR=10.0  $4.64M
  Nestor Cortes             SP   xWAR=10.0  $5.45M
  Garrett Crochet           SP   xWAR=10.0  $0.71M
  Erick Fedde               SP

## 2) Model Parameters


In [93]:
# Team budget (in millions of $)
budget = 130  # Competitive team budget (~$250M)

# Minimum budget utilization (0.0 to 1.0)
# Forces optimizer to use at least this percentage of budget
# Higher values (e.g., 0.85) encourage spending more, especially for high-budget teams
min_budget_utilization = 0.80  # Use at least 80% of budget

# Max roster size
roster_max = 30

# Objective mode controls how we trade off WAR vs salary.
#   :pure_war          → maximize total xWAR only (original behavior)
#   :war_minus_cost    → maximize xWAR - lambda_team * total cost (rich teams more willing to overpay)
objective_mode = :pure_war

# Base cost weight (per $1M). Effective lambda scales inversely with budget so
# that higher-budget teams penalize cost less in the objective.
base_lambda = 0.02
lambda_team = base_lambda * (100.0 / budget)
println("Objective mode: ", objective_mode, "  |  lambda_team = ", lambda_team)

# Positional min/max counts for a 40-man MLB roster
# Hitters: 18-20 (3-4C, 2-3 1B, 2-3 2B, 2-3 SS, 2-3 3B, 6-9 OF, 1-2 DH, 4-6 bench)
# Pitchers: 20-22 (8-10 SP, 12-15 RP)
pos_min = Dict(
    "C"  => 2,   # Need 3-4 catchers for depth
    "1B" => 1,
    "2B" => 1,
    "3B" => 1,
    "SS" => 1,
    "LF" => 1,
    "CF" => 1,
    "RF" => 1,
    "DH" => 0,   # DH optional (can use other players)
    "SP" => 5,   # 8-10 starters for rotation + depth
    "RP" => 8,  # At least 10 relievers for depth
)

pos_max = Dict(
    "C"  => 6,
    "1B" => 5,
    "2B" => 5,
    "3B" => 5,
    "SS" => 5,
    "LF" => 6,
    "CF" => 6,
    "RF" => 6,
    "DH" => 4,
    "SP" => 7,
    "RP" => 10,
)

# Ensure every position in the data has bounds
for r in positions
    if !haskey(pos_min, r)
        @warn "No pos_min for $r, setting to 0"
        pos_min[r] = 0
    end
    if !haskey(pos_max, r)
        @warn "No pos_max for $r, setting to roster_max"
        pos_max[r] = roster_max
    end
end

println("Budget: \$$budget M")
println("Roster size: $roster_max players")
println("\nPositions available: ", join(positions, ", "))


Objective mode: pure_war  |  lambda_team = 0.015384615384615385
Budget: $130 M
Roster size: 30 players

Positions available: LF, SS, 3B, 2B, CF, 1B, RF, C, DH, RP, SP


## 3) Build Model (Layer 1 + Positional Constraints)


In [94]:
model = Model(() -> Gurobi.Optimizer(env))
set_silent(model)  # comment this out if you want Gurobi logs
@variable(model, x[p in player_ids], Bin)

# If we have locked players (existing roster), force them onto the roster.
if !isempty(locked_players)
    println("Applying locked-player constraints for $(length(locked_players)) players already on the roster.")
    for p in locked_players
        @constraint(model, x[p] == 1)
    end
end

# Cost variables for free agents (decision variables)
# Allows optimizer to determine how much to pay each free agent
# Constrained: cost_var[p] >= market_cost[p] + 0.001 and <= market_cost[p] * max_multiplier
if !isempty(locked_players)
    free_agent_ids = [p for p in player_ids if !(p in locked_players)]
else
    free_agent_ids = player_ids
end

# Calculate max cost multiplier based on budget
if budget >= 250.0
    max_cost_multiplier = 1.5  # High-budget teams can pay up to 1.5x market_cost
elseif budget >= 150.0
    max_cost_multiplier = 1.3  # Medium-budget teams can pay up to 1.3x market_cost
else
    max_cost_multiplier = 1.2  # Low-budget teams can pay up to 1.2x market_cost
end

# Create cost variables for free agents
cost_var = Dict{Int, Union{VariableRef, Float64}}()
for p in free_agent_ids
    if haskey(market_cost, p) && haskey(cost, p) && market_cost[p] > 0
        # Variable cost: >= market_cost + 0.001, <= market_cost * max_multiplier
        cost_var[p] = @variable(model, lower_bound = market_cost[p] + 0.001, upper_bound = market_cost[p] * max_cost_multiplier)
    elseif haskey(cost, p)
        # No market_cost, use fixed cost from CSV
        cost_var[p] = cost[p]
    end
end

println("Cost variables created for $(length(cost_var)) free agents")
println("Max cost multiplier: $max_cost_multiplier (budget: \$$budget M)")

# Objective: choose between pure WAR or WAR-minus-cost
# Uses cost_var (variable) for free agents, market_cost (fixed) for incumbents
if objective_mode == :pure_war
    # Objective with budget utilization incentive
    if !isempty(locked_players)
        total_cost_var = sum(x[p] * market_cost[p] for p in locked_players) +
                         sum(x[p] * (haskey(cost_var, p) ? cost_var[p] : cost[p]) for p in player_ids if !(p in locked_players))
    else
        total_cost_var = sum(x[p] * (haskey(cost_var, p) ? cost_var[p] : cost[p]) for p in player_ids)
    end
    
    lambda_budget = 0.1 * (budget / 200.0)  # Scale with budget
    budget_utilization_bonus = lambda_budget * (total_cost_var / budget)
    
    @objective(model, Max, 
        sum(x[p] * xwar[p] for p in player_ids) + budget_utilization_bonus
    )
elseif objective_mode == :war_minus_cost
    # Objective with budget utilization incentive
    if !isempty(locked_players)
        total_cost_var = sum(x[p] * market_cost[p] for p in locked_players) +
                         sum(x[p] * (haskey(cost_var, p) ? cost_var[p] : cost[p]) for p in player_ids if !(p in locked_players))
    else
        total_cost_var = sum(x[p] * (haskey(cost_var, p) ? cost_var[p] : cost[p]) for p in player_ids)
    end
    
    lambda_budget = 0.1 * (budget / 200.0)  # Scale with budget
    budget_utilization_bonus = lambda_budget * (total_cost_var / budget)
    
    @objective(model, Max,
        sum(x[p] * xwar[p] for p in player_ids) -
        lambda_team * total_cost_var +
        budget_utilization_bonus
    )
else
    error("Unknown objective_mode: $objective_mode")
end

# Budget constraint:
#   - Incumbent players (locked): use market_cost (their actual contract values)
#   - Free agents: use cost_var (variable, determined by optimizer)
if !isempty(locked_players)
    @constraint(model, sum(x[p] * market_cost[p] for p in locked_players) +
                         sum(x[p] * (haskey(cost_var, p) ? cost_var[p] : cost[p]) for p in player_ids if !(p in locked_players)) <= budget)
else
    @constraint(model, sum(x[p] * (haskey(cost_var, p) ? cost_var[p] : cost[p]) for p in player_ids) <= budget)
end

# Minimum budget utilization constraint
if min_budget_utilization > 0.0
    if !isempty(locked_players)
        total_cost_for_min = sum(x[p] * market_cost[p] for p in locked_players) +
                              sum(x[p] * (haskey(cost_var, p) ? cost_var[p] : cost[p]) for p in player_ids if !(p in locked_players))
    else
        total_cost_for_min = sum(x[p] * (haskey(cost_var, p) ? cost_var[p] : cost[p]) for p in player_ids)
    end
    @constraint(model, total_cost_for_min >= min_budget_utilization * budget)
    println("Minimum budget utilization: $(round(min_budget_utilization * 100, digits=1))% (\$$(round(min_budget_utilization * budget, digits=1))M)")
end

# Roster size constraint
if !isempty(locked_players)
    num_incumbents = length(locked_players)
    @constraint(model, sum(x[p] for p in player_ids) <= roster_max)
    println("Roster constraint: $num_incumbents incumbents + up to $(roster_max - num_incumbents) free agents = $roster_max total")
else
    @constraint(model, sum(x[p] for p in player_ids) <= roster_max)
end

# Positional constraints
for r in positions
    incumbent_at_pos = length([p for p in locked_players if player_pos[p] == r])
    @constraint(model, sum(x[p] for p in players_by_pos[r]) >= pos_min[r])
    @constraint(model, sum(x[p] for p in players_by_pos[r]) <= pos_max[r])
    if incumbent_at_pos > 0
        println("  Position $r: $incumbent_at_pos incumbents, max $(pos_max[r] - incumbent_at_pos) free agents (total max: $(pos_max[r]))")
    end
end


Applying locked-player constraints for 20 players already on the roster.
Cost variables created for 248 free agents
Max cost multiplier: 1.2 (budget: $130 M)
Minimum budget utilization: 80.0% ($104.0M)
Roster constraint: 20 incumbents + up to 10 free agents = 30 total
  Position LF: 3 incumbents, max 3 free agents (total max: 6)
  Position SS: 2 incumbents, max 3 free agents (total max: 5)
  Position 2B: 2 incumbents, max 3 free agents (total max: 5)
  Position CF: 1 incumbents, max 5 free agents (total max: 6)
  Position 1B: 1 incumbents, max 4 free agents (total max: 5)
  Position RF: 1 incumbents, max 5 free agents (total max: 6)
  Position C: 1 incumbents, max 5 free agents (total max: 6)
  Position RP: 4 incumbents, max 6 free agents (total max: 10)
  Position SP: 5 incumbents, max 2 free agents (total max: 7)


In [95]:
optimize!(model)

println("Solver Status: ", termination_status(model))

if termination_status(model) != MOI.OPTIMAL &&
   termination_status(model) != MOI.LOCALLY_OPTIMAL
    error("Solver did not find an optimal solution.")
end

total_war  = objective_value(model)
# Calculate total cost: incumbents use market_cost, free agents use cost_var
if !isempty(locked_players)
    total_cost = sum(market_cost[p] * value(x[p]) for p in locked_players) +
                 sum((haskey(cost_var, p) ? value(cost_var[p]) : cost[p]) * value(x[p]) for p in player_ids if !(p in locked_players) && value(x[p]) > 0.5)
else
    total_cost = sum(cost[p] * value(x[p]) for p in player_ids)
end

println("\n" * "="^70)
println("   OPTIMAL 2025 ROSTER")
println("="^70)
println("\nTotal Projected xWAR: ", round(total_war, digits=2))
println("Total Payroll: \$", round(total_cost, digits=2), "M")
println("Budget Remaining: \$", round(budget - total_cost, digits=2), "M")

# Group selected players by position
selected = [p for p in player_ids if value(x[p]) > 0.5]
position_groups = Dict{String, Vector{Int}}()
for p in selected
    pos = player_pos[p]
    if !haskey(position_groups, pos)
        position_groups[pos] = []
    end
    push!(position_groups[pos], p)
end

# Print by position group
println("\n" * "-"^70)
println("HITTERS")
println("-"^70)
    # Check if actual_war_2025 is available
    has_actual_war = "actual_war_2025" ∈ names(players_df) && !isempty(actual_war_2025)
    
    if has_actual_war
        println(rpad("Player", 25), rpad("Pos", 5), rpad("xWAR", 10), rpad("Actual", 10), "Cost")
    else
        println(rpad("Player", 25), rpad("Pos", 5), rpad("xWAR", 10), "Cost")
    end
println("-"^70)

hitter_positions = ["C", "1B", "2B", "SS", "3B", "LF", "CF", "RF", "DH"]
hitter_war = 0.0
hitter_cost = 0.0
for pos in hitter_positions
    if haskey(position_groups, pos)
        for p in sort(position_groups[pos], by=p -> -xwar[p])
            if has_actual_war && haskey(actual_war_2025, p) && !ismissing(actual_war_2025[p])
            # Display: market_cost for incumbents, cost for free agents
            # Display: market_cost for incumbents, cost_var (or cost) for free agents
            display_cost = p in locked_players && haskey(market_cost, p) ? market_cost[p] : (haskey(cost_var, p) && isa(cost_var[p], VariableRef) ? value(cost_var[p]) : cost[p])
            println(rpad(player_name[p], 25), rpad(pos, 5), rpad(round(xwar[p], digits=2), 10), rpad(round(actual_war_2025[p], digits=2), 10), "\$$(round(display_cost, digits=2))M")
            else
            # Display: market_cost for incumbents, cost for free agents
            # Display: market_cost for incumbents, cost_var (or cost) for free agents
            display_cost = p in locked_players && haskey(market_cost, p) ? market_cost[p] : (haskey(cost_var, p) && isa(cost_var[p], VariableRef) ? value(cost_var[p]) : cost[p])
            println(rpad(player_name[p], 25), rpad(pos, 5), rpad(round(xwar[p], digits=2), 10), "\$$(round(display_cost, digits=2))M")
            end
            hitter_war += xwar[p]
            # Use market_cost for incumbents, cost for free agents
            if p in locked_players && haskey(market_cost, p)
                hitter_cost += market_cost[p]
            else
                hitter_cost += cost[p]
            end
        end
    end
end
println("-"^70)
println(rpad("Hitters Total", 25), rpad("", 5), rpad(round(hitter_war, digits=2), 10), "\$$(round(hitter_cost, digits=2))M")

println("\n" * "-"^70)
println("PITCHERS")
println("-"^70)
println(rpad("Player", 25), rpad("Pos", 5), rpad("xWAR", 10), "Cost")
println("-"^70)

pitcher_positions = ["SP", "RP"]
pitcher_war = 0.0
pitcher_cost = 0.0
for pos in pitcher_positions
    if haskey(position_groups, pos)
        for p in sort(position_groups[pos], by=p -> -xwar[p])
            if has_actual_war && haskey(actual_war_2025, p) && !ismissing(actual_war_2025[p])
            # Display: market_cost for incumbents, cost for free agents
            # Display: market_cost for incumbents, cost_var (or cost) for free agents
            display_cost = p in locked_players && haskey(market_cost, p) ? market_cost[p] : (haskey(cost_var, p) && isa(cost_var[p], VariableRef) ? value(cost_var[p]) : cost[p])
            println(rpad(player_name[p], 25), rpad(pos, 5), rpad(round(xwar[p], digits=2), 10), rpad(round(actual_war_2025[p], digits=2), 10), "\$$(round(display_cost, digits=2))M")
            else
            # Display: market_cost for incumbents, cost for free agents
            # Display: market_cost for incumbents, cost_var (or cost) for free agents
            display_cost = p in locked_players && haskey(market_cost, p) ? market_cost[p] : (haskey(cost_var, p) && isa(cost_var[p], VariableRef) ? value(cost_var[p]) : cost[p])
            println(rpad(player_name[p], 25), rpad(pos, 5), rpad(round(xwar[p], digits=2), 10), "\$$(round(display_cost, digits=2))M")
            end
            pitcher_war += xwar[p]
            # Use market_cost for incumbents, cost for free agents
            if p in locked_players && haskey(market_cost, p)
                pitcher_cost += market_cost[p]
            else
                pitcher_cost += cost[p]
            end
        end
    end
end
println("-"^70)
println(rpad("Pitchers Total", 25), rpad("", 5), rpad(round(pitcher_war, digits=2), 10), "\$$(round(pitcher_cost, digits=2))M")

println("\n" * "="^70)
println("ROSTER SUMMARY")
println("="^70)
println("Total Players: $(length(selected))")
println("Hitters: $(sum(length(get(position_groups, p, [])) for p in hitter_positions))")
println("Pitchers: $(sum(length(get(position_groups, p, [])) for p in pitcher_positions))")
println("\nProjected Team xWAR: $(round(total_war, digits=2))")
println("Total Payroll: \$$(round(total_cost, digits=2))M")
println("="^70)


Solver Status: OPTIMAL

   OPTIMAL 2025 ROSTER

Total Projected xWAR: 121.24
Total Payroll: $130.0M
Budget Remaining: $0.0M

----------------------------------------------------------------------
HITTERS
----------------------------------------------------------------------
Player                   Pos  xWAR      Actual    Cost
----------------------------------------------------------------------
Freddy Fermin            C    2.12      2.86      $0.94M
Connor Wong              C    1.05      -1.52     $0.79M
Triston Casas            1B   0.45      -4.75     $0.77M
Romy Gonzalez            2B   0.61      3.22      $0.77M
Nick Sogard              2B   0.55      0.61      $0.76M
David Hamilton           SS   2.78      1.73      $0.78M
Trevor Story             SS   1.46      3.89      $22.5M
Jose Iglesias            3B   2.74      -0.95     $3.0M
Juan Soto                LF   7.21      6.32      $50.4M
Wilyer Abreu             LF   2.86      4.48      $0.78M
Ceddanne Rafaela         LF   

## Free Agents Signed

This section shows the free agents that were signed in the optimal roster.

In [98]:
# Identify free agents that were signed (selected but not locked)
free_agents_signed = [p for p in selected if !(p in locked_players)]

if !isempty(free_agents_signed)
    println("="^70)
    println("   FREE AGENTS SIGNED")
    println("="^70)
    println()
    
    # Sort by xWAR descending
    free_agents_sorted = sort(free_agents_signed, by=p -> -xwar[p])
    
    # Print header
    println(rpad("Player", 30), rpad("Pos", 5), rpad("xWAR", 8), rpad("Cost", 10), rpad("Market Cost", 12), "Value Gap")
    println("-"^70)
    
    total_fa_cost = 0.0
    total_fa_war = 0.0
    
    for p in free_agents_sorted
        # Use cost_var (variable cost) for free agents, fallback to cost if not available
        fa_cost = haskey(cost_var, p) && isa(cost_var[p], VariableRef) ? value(cost_var[p]) : (haskey(cost, p) ? cost[p] : 0.0)
        fa_market_cost = haskey(market_cost, p) ? market_cost[p] : 0.0
        value_gap = fa_cost - fa_market_cost
        
        total_fa_cost += fa_cost
        total_fa_war += xwar[p]
        
        println(rpad(player_name[p], 30), rpad(player_pos[p], 5), 
                rpad(round(xwar[p], digits=2), 8), 
                rpad("\$$(round(fa_cost, digits=2))M", 10),
                rpad("\$$(round(fa_market_cost, digits=2))M", 12),
                "\$$(round(value_gap, digits=2))M")
    end
    
    println("-"^70)
    println(rpad("Total", 30), rpad("", 5), rpad(round(total_fa_war, digits=2), 8), 
            rpad("\$$(round(total_fa_cost, digits=2))M", 10), rpad("", 12), "")
    println()
    println("Number of free agents signed: $(length(free_agents_signed))")
    println("Total free agent cost: \$$(round(total_fa_cost, digits=2))M")
    println("Total free agent xWAR: $(round(total_fa_war, digits=2))")
    
    if has_actual_war
        total_fa_actual_war = sum(actual_war_2025[p] for p in free_agents_signed if haskey(actual_war_2025, p) && !ismissing(actual_war_2025[p]))
        println("Total free agent actual WAR (2025): $(round(total_fa_actual_war, digits=2))")
        println("Prediction error: $(round(total_fa_war - total_fa_actual_war, digits=2)) WAR")
    end
    
    println("="^70)
else
    println("="^70)
    println("   FREE AGENTS SIGNED")
    println("="^70)
    println()
    println("No free agents were signed in the optimal roster.")
    println("="^70)
end


   FREE AGENTS SIGNED

Player                        Pos  xWAR    Cost      Market Cost Value Gap
----------------------------------------------------------------------
Taj Bradley                   SP   10.0    $0.78M    $0.77M      $0.0M
Nick Pivetta                  SP   10.0    $1.0M     $1.0M       $0.0M
Luis Ortiz                    RP   7.29    $0.78M    $0.78M      $0.0M
Juan Soto                     LF   7.21    $50.4M    $46.88M     $3.52M
Fernando Cruz                 RP   6.03    $0.79M    $0.78M      $0.0M
Scott Blewett                 RP   5.4     $0.76M    $0.76M      $0.0M
Nick Mears                    RP   4.97    $0.96M    $0.96M      $0.0M
Mark Leiter Jr.               RP   4.77    $2.05M    $2.05M      $0.0M
Jose Iglesias                 3B   2.74    $3.0M     $3.0M       $0.0M
Freddy Fermin                 C    2.12    $0.94M    $0.78M      $0.16M
----------------------------------------------------------------------
Total                              60.53   $61.4